Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### logistic

In [16]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

lam = 0.01
batch_size = 128
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+lam*tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.095901
Minibatch accuracy: 8.6%
Validation accuracy: 10.5%
Minibatch loss at step 500: 0.955065
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.709361
Minibatch accuracy: 83.6%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 0.778592
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 0.780603
Minibatch accuracy: 82.0%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.706130
Minibatch accuracy: 79.7%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.868946
Minibatch accuracy: 78.9%
Validation accuracy: 80.5%
Test accuracy: 87.5%


### neural network

In [20]:
batch_size = 128
hidden_layer_size = 1024
lam = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.add(tf.matmul(tf_train_dataset, weights1), biases1)
  hidden_layer = tf.nn.relu(hidden_layer)
  logits = tf.matmul(hidden_layer, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+lam*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
      tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights1), biases1)),
                                   weights2)
                                   +biases2)
  test_prediction = tf.nn.softmax(tf.matmul(
      tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights1), biases1)),
                                   weights2)
                                   +biases2)

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3521.118652
Minibatch accuracy: 12.5%
Validation accuracy: 33.1%
Minibatch loss at step 500: 21.315874
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Minibatch loss at step 1000: 0.850843
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.751678
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.707795
Minibatch accuracy: 85.2%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.693141
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.806857
Minibatch accuracy: 80.5%
Validation accuracy: 83.3%
Test accuracy: 89.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step//(batch_size*10)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.398300
Minibatch accuracy: 9.4%
Validation accuracy: 10.6%
Minibatch loss at step 500: 0.322308
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 1000: 0.128625
Minibatch accuracy: 100.0%
Validation accuracy: 71.4%
Minibatch loss at step 1500: 0.126608
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 2000: 0.126276
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 2500: 0.126092
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 3000: 0.127906
Minibatch accuracy: 100.0%
Validation accuracy: 70.8%
Test accuracy: 77.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Introduce Dropout

In [23]:
batch_size = 128
hidden_layer_size = 1024
lam = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.add(tf.matmul(tf_train_dataset, weights1), biases1)
  hidden_layer = tf.nn.relu(hidden_layer)
  hidden_layer = tf.nn.dropout(hidden_layer, 0.8)
  logits = tf.matmul(hidden_layer, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+lam*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
      tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights1), biases1)),
                                   weights2)
                                   +biases2)
  test_prediction = tf.nn.softmax(tf.matmul(
      tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights1), biases1)),
                                   weights2)
                                   +biases2)

In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3588.622070
Minibatch accuracy: 6.2%
Validation accuracy: 24.2%
Minibatch loss at step 500: 21.386503
Minibatch accuracy: 81.2%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 0.899725
Minibatch accuracy: 85.2%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.765340
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.718822
Minibatch accuracy: 85.2%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.723978
Minibatch accuracy: 82.8%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.848526
Minibatch accuracy: 79.7%
Validation accuracy: 82.9%
Test accuracy: 89.1%


### extreme overfitting case

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step / (batch_size * 10)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3536.827881
Minibatch accuracy: 13.3%
Validation accuracy: 25.7%
Minibatch loss at step 500: 21.010880
Minibatch accuracy: 100.0%
Validation accuracy: 67.3%
Minibatch loss at step 1000: 0.330974
Minibatch accuracy: 100.0%
Validation accuracy: 72.2%
Minibatch loss at step 1500: 0.180827
Minibatch accuracy: 100.0%
Validation accuracy: 72.2%
Minibatch loss at step 2000: 0.175651
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 2500: 0.171000
Minibatch accuracy: 100.0%
Validation accuracy: 72.4%
Minibatch loss at step 3000: 0.170929
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Test accuracy: 78.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [8]:
batch_size = 128
hidden_layer_size = 1024
lam = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases1 = tf.Variable(tf.zeros([hidden_layer_size]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_layer_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_layer = tf.add(tf.matmul(tf_train_dataset, weights1), biases1)
  hidden_layer = tf.nn.relu(hidden_layer)
  hidden_layer = tf.nn.dropout(hidden_layer, 0.8)
  logits = tf.matmul(hidden_layer, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+lam*(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96, staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
      tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset, weights1), biases1)),
                                   weights2)
                                   +biases2)
  test_prediction = tf.nn.softmax(tf.matmul(
      tf.nn.relu(tf.add(tf.matmul(tf_test_dataset, weights1), biases1)),
                                   weights2)
                                   +biases2)

In [9]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3587.572754
Minibatch accuracy: 14.8%
Validation accuracy: 27.7%
Minibatch loss at step 500: 21.331734
Minibatch accuracy: 82.8%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 0.904013
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 0.756955
Minibatch accuracy: 85.2%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 0.701565
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.699003
Minibatch accuracy: 81.2%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.813879
Minibatch accuracy: 82.0%
Validation accuracy: 83.4%
Minibatch loss at step 3500: 0.733187
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 0.706185
Minibatch accuracy: 83.6%
Validation accuracy: 84.3%
Minibatch loss at step 4500: 0.729590
Minibatch accuracy: 84.4%
Validation accuracy: 84.5%
Minibatch loss at step 5000: 0.896676
Minibatch accuracy: 78.9%
Validation acc